In [ ]:
from sunpy.net import Fido, attrs as a
import astropy.units as u
import sunpy.map
import astropy.io.fits
import astropy.wcs
from astropy.coordinates import SkyCoord
import matplotlib.pyplot as plt 
import os
import numpy as np

In [ ]:
tstart = '2024-05-21T10:19:05'
ic_series = 'hmi.Ic_45s'
mag_series = 'hmi.M_45s'
email = 'avijeet.prasad@astro.uio.no'
ln_min = -242.40 # (arcsec)
ln_max = -153.04 # (arcsec)
lt_min = -177.44 # (arcsec)
lt_max = -88.38  # (arcsec)

In [ ]:
ic_series.casefold()

In [ ]:
res = Fido.search(a.Time(tstart, tstart), a.jsoc.Series(ic_series), a.jsoc.Notify(email))  
ic_path = 'temp/' + 'hmi.ic_45s.' + res['jsoc'][0]['T_REC'].replace(':', '').replace('.', '') + '.2.continuum.fits'

In [ ]:
res2 = Fido.search(a.Time(tstart, tstart), a.jsoc.Series(mag_series), a.jsoc.Notify(email))  
blos_path = 'temp/' + 'hmi.m_45s.' + res2['jsoc'][0]['T_REC'].replace(':', '').replace('.', '') + '.2.fits'

In [ ]:
if not os.path.exists(ic_path):
    files = Fido.fetch(res, path='temp/', overwrite=True)
else:
    print(f'File already exists: {ic_path}')


In [ ]:
res['jsoc'][0]['T_REC'].replace(':', '').replace('.', '') + '2.continuum.fits'

In [ ]:
# check if ic_path exists using os.path.exists()
import os
os.path.exists(ic_path)

In [ ]:
files

In [ ]:
files2 = Fido.fetch(res2, path='temp/', overwrite=True)

In [ ]:
from astropy.coordinates import SkyCoord, SkyOffsetFrame
import astropy.units as u
from matplotlib.patches import Circle
from astropy.coordinates import SkyCoord


In [ ]:
import os
from sunpy.net import Fido, attrs as a
import astropy.units as u
import sunpy.map
import astropy.io.fits
import astropy.wcs
from astropy.coordinates import SkyCoord
import matplotlib.pyplot as plt

def fetch_and_prepare_data(series, email, time_start, path='temp/'):
    res = Fido.search(a.Time(time_start, time_start), a.jsoc.Series(series), a.jsoc.Notify(email))
    # check if the file already exists else download it
    time_str = res['jsoc'][0]['T_REC'].replace(':', '').replace('.', '')
    if series == 'hmi.Ic_45s':
        segment = 'continuum'
    elif series == 'hmi.M_45s':
        segment = 'magnetogram'
    file_path = f'{path}{series.casefold()}.{time_str}.2.{segment}.fits'
    if not os.path.exists(file_path):
        files = Fido.fetch(res, path=path, overwrite=False)
        print(f'File downloaded: {files[0]}')
        file_path = files[0]
    else: 
        print(f'File already exists: {file_path}')
    return file_path

def read_fits(file):
    with astropy.io.fits.open(file) as hdul:
        data = hdul[1].data
        header = hdul[1].header
    wcs = astropy.wcs.WCS(header=header)
    return sunpy.map.Map(data, wcs).rotate()

def plot_submaps(map1, map2, bottom_left, top_right, center_coord=None, draw_circle=False, radius=87, draw_rectangle=False, height=56, width=56, rot_fov=0):
    submap1 = map1.submap(bottom_left, top_right=top_right)
    submap2 = map2.submap(bottom_left, top_right=top_right)
    
    fig, axes = plt.subplots(1, 2, figsize=(16, 8), subplot_kw={'projection': submap1})
    if draw_circle:
        radius = radius * u.arcsec
        # Drawing rectangles rotated 1 degree apart to form a circle
        for angle in np.arange(0, 360, 0.5):
            rotation_angle = angle * u.deg
            offset_frame = SkyOffsetFrame(origin=center_coord, rotation=rotation_angle)
            rectangle = SkyCoord(lon=[-1/2, 1/2] * radius, lat=[-1/2, 1/2] * radius, frame=offset_frame)

            submap1.draw_quadrangle(
                rectangle,
                axes=axes[0],
                edgecolor="white",
                linestyle="--",
                linewidth=1,
            )
            submap2.draw_quadrangle(
                rectangle,
                axes=axes[1],
                edgecolor="white",
                linestyle="--",
                linewidth=1,
            )
    if draw_rectangle:
        width = width * u.arcsec
        height = height * u.arcsec
        rotation_angle = rot_fov * u.deg
        offset_frame = SkyOffsetFrame(origin=center_coord, rotation=rotation_angle)
        rectangle = SkyCoord(lon=[-1/2, 1/2] * width, lat=[-1/2, 1/2] * height, frame=offset_frame)
        
        submap1.draw_quadrangle(
            rectangle,
            axes=axes[0],
            edgecolor="red",
            linestyle="--",
            linewidth=1,
        )
        submap2.draw_quadrangle(
            rectangle,
            axes=axes[1],
            edgecolor="red",
            linestyle="--",
            linewidth=1,
        )
    title_pad=5
    submap1.plot(axes=axes[0])
    axes[0].set_title(axes[0].get_title(), pad=title_pad)
    axes[0].grid(False)
    
    submap2.plot(axes=axes[1])
    axes[1].set_title(axes[1].get_title(), pad=title_pad)
    axes[1].grid(False)
    
    plt.tight_layout()
    plt.show()

def plot_hmi_ic_mag(tstart, ic_series, mag_series, email, x1, x2, y1, y2, draw_circle=False, radius=87, draw_rectangle=False, height=56, width=56, rot_fov=0):

    ic_file = fetch_and_prepare_data(ic_series, email, tstart)
    blos_file = fetch_and_prepare_data(mag_series, email, tstart)
    
    ic_map = read_fits(ic_file)
    blos_map = read_fits(blos_file)
    
    center_coord = SkyCoord((x1 + x2) / 2 * u.arcsec, (y1 + y2) / 2 * u.arcsec, frame=ic_map.coordinate_frame)
    top_right = SkyCoord(x2 * u.arcsec, y2 * u.arcsec, frame=ic_map.coordinate_frame)
    bottom_left = SkyCoord(x1 * u.arcsec, y1 * u.arcsec, frame=ic_map.coordinate_frame)
    
    plot_submaps(ic_map, blos_map, bottom_left, top_right, center_coord, draw_circle, radius, draw_rectangle, height, width, rot_fov)

def main():
    # Example usage
    tstart = '2024-05-21T10:19:05'
    ic_series = 'hmi.Ic_45s'
    mag_series = 'hmi.M_45s'
    email = 'avijeet.prasad@astro.uio.no'
    x1 = -242.40 
    x2 = -149.50 
    y1 = -177.44 
    y2 = -87.83  
    plot_hmi_ic_mag(tstart, ic_series, mag_series, email, x1, x2, y1, y2, draw_rectangle=True, rot_fov=0, width=56, height=56, draw_circle=False, radius=87)

if __name__ == "__main__":
    main()


In [ ]:
import subprocess
import os
import tempfile

def run_enhance_script(python_path, script_dir, input_file, file_type, output_file, debug_mode=False):
    """
    Run the enhance.py script with the specified parameters.

    Parameters:
    - python_path (str): Path to the Python interpreter in the conda environment.
    - script_dir (str): Directory containing the enhance.py script.
    - input_file (str): Path to the input file.
    - file_type (str): Type of the file (e.g., 'intensity', 'blos').
    - output_file (str): Path to the output file.
    - debug_mode (bool): If True, print the command instead of running it.
    """
    # Check if the Python interpreter exists
    if not os.path.exists(python_path):
        print(f"Error: Python interpreter '{python_path}' does not exist.")
        return

    # Check if the script directory exists
    if not os.path.exists(script_dir):
        print(f"Error: Script directory '{script_dir}' does not exist.")
        return

    # Check if the input file exists
    if not os.path.exists(input_file):
        print(f"Error: Input file '{input_file}' does not exist.")
        return

    # Construct the shell script content
    script_content = f"""#!/bin/bash
cd {script_dir}
{python_path} enhance.py -i "{input_file}" -t "{file_type}" -o "{output_file}"
"""

    # Create a temporary file to hold the shell script
    with tempfile.NamedTemporaryFile('w', delete=False, suffix=".sh") as temp_script:
        temp_script.write(script_content)
        temp_script_path = temp_script.name

    # Make the temporary script executable
    os.chmod(temp_script_path, 0o755)

    # Construct the command
    command = [temp_script_path]

    if debug_mode:
        # Print the command instead of running it
        print("Debug Mode: The following command would be executed:")
        print(" ".join(command))
        print("Script content:")
        print(script_content)
    else:
        # Run the temporary shell script
        result = subprocess.run(command, capture_output=True, text=True)
        
        # Print the output and error messages
        print("STDOUT:", result.stdout)
        print("STDERR:", result.stderr)
    
    # Clean up the temporary script
    os.remove(temp_script_path)

    print("Enhanced file saved at:", output_file)
    return output_file
# Example usage
python_path = "/mn/stornext/d9/data/avijeetp/envs/enhance/bin/python"
script_dir = "/mn/stornext/u3/avijeetp/codes/enhance"
input_file = "/mn/stornext/u3/avijeetp/codes/pyMilne/example_CRISP/temp/hmi.ic_45s.20200807_082230_TAI.2.continuum.fits"
file_type = "intensity"
output_file = "/mn/stornext/u3/avijeetp/codes/pyMilne/example_CRISP/temp/enhanced_hmi.ic_45s.20200807_082230_TAI.2.continuum.fits"
debug_mode = False

run_enhance_script(python_path, script_dir, input_file, file_type, output_file, debug_mode)